<a href="https://colab.research.google.com/github/prateekbeh/algo_ds/blob/master/seismic_lwe2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset as dt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.dates as mdt
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
from datetime import datetime
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.animation as anim
from shapely.geometry import Point , shape , Polygon
from dataclasses import dataclass, fields
import matplotlib.pyplot as pl
import hvplot.xarray
#import fiona
import shapefile
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import netCDF4 as nc
import math
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from datetime import date
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point

ModuleNotFoundError: No module named 'netCDF4'

In [ ]:
!pip install rasterio

In [11]:

from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']
catalog_data = catalog_data[catalog_data['mag'] >= 5]


# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.5
catalog_data['lat_grid'] = np.where((catalog_data['latitude'] % 1) < 0.5,
                                    np.floor(catalog_data['latitude']) + 0.25,
                                    np.floor(catalog_data['latitude']) + 0.75)

catalog_data['lon_grid'] = np.where((catalog_data['longitude'] % 1) < 0.5,
                                     np.floor(catalog_data['longitude']) + 0.25,
                                     np.floor(catalog_data['longitude']) + 0.75)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-11-dcca60911d15>:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


In [12]:
# prompt: iterate through the catalog data and implement a formula. Given the mag(in mw) , obtain D_eps = 10^(0.442M-2.73)/(eps)^1/3 , where eps = 10^-8

# Assuming 'mag' column exists in your catalog_data DataFrame

eps = 10**-8
catalog_data['D_eps'] = 10**(0.442 * catalog_data['mag'] - 2.73) / (eps**(1/3))

In [35]:
# prompt: Implement one more formula, for catalog data, i.e. log (D_eps * T) = 0.63 * mag +- 0.15, get the value of T. the log is base 10

# Calculate T using the formula log(D_eps * T) = 0.63 * mag +- 0.15
# We'll consider both the + and - cases for the uncertainty

catalog_data['T_plus'] = 10**(0.63 * catalog_data['mag'] + 0.15) / catalog_data['D_eps']
catalog_data['T_minus'] = 10**(0.63 * catalog_data['mag'] - 0.15) / catalog_data['D_eps']

# Calculate the mean of T
catalog_data['T'] = (catalog_data['T_plus'] + catalog_data['T_minus']) / 2

In [36]:
catalog_data

,date,time,latitude,longitude,depth,mag,magtype,lat_grid,lon_grid,D_eps,T,T_minus,T_plus
27321,2002-06-17,16:55:07.680Z,40.809833,-124.552000,17.195,5.20,mw,40.75,-124.75,171.817212,11.650329,7.779171,15.521487
27743,2003-12-22,19:15:56.240Z,35.700500,-121.100500,8.382,6.50,mw,35.75,-121.25,645.158861,20.452114,13.656309,27.247919
28206,2004-09-18,23:02:17.750Z,38.009500,-118.678500,3.355,5.40,mw,38.25,-118.75,210.604025,12.703933,8.482685,16.925181
28210,2004-09-18,23:43:41.490Z,38.023333,-118.658000,0.431,5.35,mw,38.25,-118.75,200.155103,12.431920,8.301056,16.562784
28244,2004-09-28,17:15:24.250Z,35.818167,-120.366000,8.136,5.97,mw,35.75,-120.25,376.189497,16.259164,10.856587,21.661740
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37449,2022-10-25,18:42:02.710Z,37.311500,-121.672000,8.380,5.06,mw,37.25,-121.75,149.000137,10.965241,7.321724,14.608759
37482,2022-12-20,10:34:24.770Z,40.525000,-124.423000,17.910,6.40,mw,40.75,-124.25,582.729047,19.585660,13.077759,26.093560
37605,2023-05-11,23:19:41.990Z,40.204167,-121.109500,5.850,5.48,mw,40.25,-121.25,228.468677,13.151588,8.781594,17.521583
37607,2023-05-12,10:18:41.310Z,40.196000,-121.099833,6.060,5.16,mw,40.25,-121.25,164.963046,11.450335,7.645631,15.255039


In [37]:
# prompt: convert T_plus to months

# Convert T_plus to months
catalog_data['T_plus_months'] = catalog_data['T_plus'] / 30.44 # Approximate days in a month

In [39]:
# prompt: show all the rows of catalog_data

pd.set_option("display.max_rows", None, "display.max_columns", None)
catalog_data

,date,time,latitude,longitude,depth,mag,magtype,lat_grid,lon_grid,D_eps,T,T_minus,T_plus,T_plus_months
27321,2002-06-17,16:55:07.680Z,40.809833,-124.552000,17.195,5.20,mw,40.75,-124.75,171.817212,11.650329,7.779171,15.521487,0.509904
27743,2003-12-22,19:15:56.240Z,35.700500,-121.100500,8.382,6.50,mw,35.75,-121.25,645.158861,20.452114,13.656309,27.247919,0.895135
28206,2004-09-18,23:02:17.750Z,38.009500,-118.678500,3.355,5.40,mw,38.25,-118.75,210.604025,12.703933,8.482685,16.925181,0.556018
28210,2004-09-18,23:43:41.490Z,38.023333,-118.658000,0.431,5.35,mw,38.25,-118.75,200.155103,12.431920,8.301056,16.562784,0.544112
28244,2004-09-28,17:15:24.250Z,35.818167,-120.366000,8.136,5.97,mw,35.75,-120.25,376.189497,16.259164,10.856587,21.661740,0.711621
28260,2004-09-29,17:10:04.030Z,35.953667,-120.502167,10.772,5.00,mw,35.75,-120.75,140.173742,10.684106,7.134004,14.234209,0.467615
28261,2004-09-29,22:54:54.240Z,35.389833,-118.623500,2.788,5.03,mw,35.25,-118.75,144.519572,10.823761,7.227254,14.420268,0.473728
28485,2005-06-12,15:41:46.520Z,33.532500,-116.566667,13.051,5.20,mw,33.75,-116.75,171.817212,11.650329,7.779171,15.521487,0.509904
28573,2005-09-02,01:27:19.590Z,33.153333,-115.646333,1.207,5.08,mw,33.25,-115.75,152.064090,11.060588,7.385388,14.735787,0.484093
28815,2006-05-24,04:20:26.010Z,32.306667,-115.227833,6.006,5.37,mw,32.25,-115.25,204.270977,12.540019,8.373236,16.706802,0.548844


In [40]:
# prompt: divide D_eps by 55.5 and add it as another column

catalog_data['D_eps_divided'] = catalog_data['D_eps'] / 55.5

In [43]:
# prompt: round D_eps_divided to integer.

catalog_data['D_eps_divided_rounded'] = catalog_data['D_eps_divided'].round().astype(int)

In [45]:
# prompt: add year_month in the dataframe from date

# Extract year and month from the 'date' column
catalog_data['year'] = catalog_data['date'].dt.year
catalog_data['month'] = catalog_data['date'].dt.month

# Create the 'year_month' column by combining year and month
catalog_data['year_month'] = catalog_data['date'].dt.strftime('%Y-%m')

In [48]:
# prompt: add four columns to catalog_data . 1.) lat_min = lat_grid - 0.5 * D_eps_divided_rounded  , 2.)lat_max =  lat_grid + 0.5 * D_eps_divided_rounded , 3.) lon_min =  lon_grid - 0.5 * D_eps_divided_rounded , 4.) lon_max = lon_grid + 0.5 * D_eps_divided_rounded

# Calculate lat_min, lat_max, lon_min, and lon_max
catalog_data['lat_min'] = catalog_data['lat_grid'] - 0.5 * catalog_data['D_eps_divided_rounded']
catalog_data['lat_max'] = catalog_data['lat_grid'] + 0.5 * catalog_data['D_eps_divided_rounded']
catalog_data['lon_min'] = catalog_data['lon_grid'] - 0.5 * catalog_data['D_eps_divided_rounded']
catalog_data['lon_max'] = catalog_data['lon_grid'] + 0.5 * catalog_data['D_eps_divided_rounded']

In [ ]:
# prompt: for a particular lat_min in each row, add [ lat_min , lon_min + 0.5 ]  until lon_min + 0.5 < lon_max.

# Create an empty list to store the new coordinates
new_coordinates = []

# Iterate through each row of the DataFrame
for index, row in catalog_data.iterrows():
    lat_min = row['lat_min']
    lat_max = row['lat_max']
    lon_min = row['lon_min']
    lon_max = row['lon_max']

    # Create a list to store coordinates for the current row
    row_coordinates = []

    # Add coordinates until lon_min + 0.5 < lon_max
    while lat_min + 0.5 < lat_max:
      while lon_min + 0.5 < lon_max:
          row_coordinates.append([lat_min, lon_min + 0.5])
          lon_min += 0.5
          new_coordinates.append(row_coordinates)
      lat_min += 0.5
# Add the new coordinates as a new column to the DataFrame
catalog_data['new_coords'] = new_coordinates


In [ ]:
# prompt: for a particular lat_min in each row, add [ lat_min , lon_min + 0.5 ]  until lon_min + 0.5 < lon_max.

# Create an empty list to store the new coordinates
new_coordinates = []

# Iterate through each row of the DataFrame
for index, row in catalog_data.iterrows():
    lat_min = row['lat_min']
    lon_min = row['lon_min']
    lon_max = row['lon_max']

    # Create a list to store coordinates for the current row
    row_coordinates = []

    # Add coordinates until lon_min + 0.5 < lon_max
    while lon_min + 0.5 < lon_max:
        row_coordinates.append([lat_min, lon_min + 0.5])
        lon_min += 0.5

    new_coordinates.append(row_coordinates)

# Add the new coordinates as a new column to the DataFrame
catalog_data['new_coords'] = new_coordinates


In [56]:
# prompt: show all the values of new_coords

# Assuming the code you provided is already executed and 'catalog_data' DataFrame is available.

# Print all values of 'new_coords'
for index, row in catalog_data.iterrows():
    print(f"Index: {index}, new_coords: {row['new_coords']}")

Index: 27321, new_coords: [[39.25, -125.75], [39.25, -125.25], [39.25, -124.75], [39.25, -124.25], [39.25, -123.75]]
Index: 27743, new_coords: [[29.75, -126.75], [29.75, -126.25], [29.75, -125.75], [29.75, -125.25], [29.75, -124.75], [29.75, -124.25], [29.75, -123.75], [29.75, -123.25], [29.75, -122.75], [29.75, -122.25], [29.75, -121.75], [29.75, -121.25], [29.75, -120.75], [29.75, -120.25], [29.75, -119.75], [29.75, -119.25], [29.75, -118.75], [29.75, -118.25], [29.75, -117.75], [29.75, -117.25], [29.75, -116.75], [29.75, -116.25], [29.75, -115.75]]
Index: 28206, new_coords: [[36.25, -120.25], [36.25, -119.75], [36.25, -119.25], [36.25, -118.75], [36.25, -118.25], [36.25, -117.75], [36.25, -117.25]]
Index: 28210, new_coords: [[36.25, -120.25], [36.25, -119.75], [36.25, -119.25], [36.25, -118.75], [36.25, -118.25], [36.25, -117.75], [36.25, -117.25]]
Index: 28244, new_coords: [[32.25, -123.25], [32.25, -122.75], [32.25, -122.25], [32.25, -121.75], [32.25, -121.25], [32.25, -120.75], [

In [55]:
catalog_data

,date,time,latitude,longitude,depth,mag,magtype,lat_grid,lon_grid,D_eps,T,T_minus,T_plus,T_plus_months,D_eps_divided,D_eps_divided_rounded,year,month,year_month,lat_min,lat_max,lon_min,lon_max,coordinates,new_coords
27321,2002-06-17,16:55:07.680Z,40.809833,-124.552000,17.195,5.20,mw,40.75,-124.75,171.817212,11.650329,7.779171,15.521487,0.509904,3.095806,3,2002,6,2002-06,39.25,42.25,-126.25,-123.25,"[[39.75, -125.75], [40.25, -125.25], [40.75, -...","[[39.25, -125.75], [39.25, -125.25], [39.25, -..."
27743,2003-12-22,19:15:56.240Z,35.700500,-121.100500,8.382,6.50,mw,35.75,-121.25,645.158861,20.452114,13.656309,27.247919,0.895135,11.624484,12,2003,12,2003-12,29.75,41.75,-127.25,-115.25,[],"[[29.75, -126.75], [29.75, -126.25], [29.75, -..."
28206,2004-09-18,23:02:17.750Z,38.009500,-118.678500,3.355,5.40,mw,38.25,-118.75,210.604025,12.703933,8.482685,16.925181,0.556018,3.794667,4,2004,9,2004-09,36.25,40.25,-120.75,-116.75,[],"[[36.25, -120.25], [36.25, -119.75], [36.25, -..."
28210,2004-09-18,23:43:41.490Z,38.023333,-118.658000,0.431,5.35,mw,38.25,-118.75,200.155103,12.431920,8.301056,16.562784,0.544112,3.606398,4,2004,9,2004-09,36.25,40.25,-120.75,-116.75,[],"[[36.25, -120.25], [36.25, -119.75], [36.25, -..."
28244,2004-09-28,17:15:24.250Z,35.818167,-120.366000,8.136,5.97,mw,35.75,-120.25,376.189497,16.259164,10.856587,21.661740,0.711621,6.778189,7,2004,9,2004-09,32.25,39.25,-123.75,-116.75,[],"[[32.25, -123.25], [32.25, -122.75], [32.25, -..."
28260,2004-09-29,17:10:04.030Z,35.953667,-120.502167,10.772,5.00,mw,35.75,-120.75,140.173742,10.684106,7.134004,14.234209,0.467615,2.525653,3,2004,9,2004-09,34.25,37.25,-122.25,-119.25,[],"[[34.25, -121.75], [34.25, -121.25], [34.25, -..."
28261,2004-09-29,22:54:54.240Z,35.389833,-118.623500,2.788,5.03,mw,35.25,-118.75,144.519572,10.823761,7.227254,14.420268,0.473728,2.603956,3,2004,9,2004-09,33.75,36.75,-120.25,-117.25,[],"[[33.75, -119.75], [33.75, -119.25], [33.75, -..."
28485,2005-06-12,15:41:46.520Z,33.532500,-116.566667,13.051,5.20,mw,33.75,-116.75,171.817212,11.650329,7.779171,15.521487,0.509904,3.095806,3,2005,6,2005-06,32.25,35.25,-118.25,-115.25,[],"[[32.25, -117.75], [32.25, -117.25], [32.25, -..."
28573,2005-09-02,01:27:19.590Z,33.153333,-115.646333,1.207,5.08,mw,33.25,-115.75,152.064090,11.060588,7.385388,14.735787,0.484093,2.739894,3,2005,9,2005-09,31.75,34.75,-117.25,-114.25,[],"[[31.75, -116.75], [31.75, -116.25], [31.75, -..."
28815,2006-05-24,04:20:26.010Z,32.306667,-115.227833,6.006,5.37,mw,32.25,-115.25,204.270977,12.540019,8.373236,16.706802,0.548844,3.680558,4,2006,5,2006-05,30.25,34.25,-117.25,-113.25,[],"[[30.25, -116.75], [30.25, -116.25], [30.25, -..."


In [20]:
# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]

    # Convert longitudes from [0, 360] to [-180, 180] range
    longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 31, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)


Variables in GRACE Mascon NetCDF file:
dict_keys(['lon', 'lat', 'time', 'land_mask', 'lwe_thickness', 'lon_bounds', 'lat_bounds', 'time_bounds'])
LWE Thickness Data Shape: (231, 360, 720)
Latitude Data Shape: (360,)
Longitude Data Shape: (720,)
Time Data Shape: (231,)


In [21]:
grace_data

,year_month,latitude,longitude,lwe_thickness
0,2002-04,31.25,-124.75,-0.967506
1,2002-04,31.25,-124.25,-0.967506
2,2002-04,31.25,-123.75,-1.035457
3,2002-04,31.25,-123.25,-1.035457
4,2002-04,31.25,-122.75,-1.035457
...,...,...,...,...
111799,2024-03,41.75,-116.25,-5.165121
111800,2024-03,41.75,-115.75,-4.416625
111801,2024-03,41.75,-115.25,-3.472987
111802,2024-03,41.75,-114.75,-2.648243


In [31]:
for index, row in catalog_data.iterrows():
    D_eps_value = row['D_eps']
    lat_grid_value = row['lat_grid']
    lon_grid_value = row['lon_grid']
    no_grid_points = D_eps_value / 50;
    print(no_grid_points)
    grace_data.loc[(grace_data['latitude'] == lat_grid_value) & (grace_data['longitude'] == lon_grid_value), 'no_grid_points'] = no_grid_points



3.4363442321638122
12.903177224581206
4.212080496450105
4.003102056751097
7.523789949383983
2.803474836693539
2.890391439652425
3.4363442321638122
3.041281795335278
4.085419545577364
4.212080496450105
2.803474836693539
4.431968472713484
3.1038209653728783
3.1038209653728783
2.8321526804790396
7.006427982172571
3.1038209653728783
3.1038209653728783
4.387091196004931
2.980002732704843
3.041281795335278
2.803474836693539
6.328439933315081
26.308534843236803
3.652722353106715
5.716058466807852
4.342668338134675
4.127210904641055
3.507007238983056
3.76596831997919
5.774530271025377
4.298695297743293
3.0723922597820246
3.8827252706530997
4.255167520063849
3.76596831997919
10.526807042184048
3.1038209653728783
5.6581787370497185
3.1038209653728783
7.916562819511961
5.833600206264177
3.804491887149453
3.401548458700596
3.0723922597820246
2.8321526804790396
5.162935058285399
5.162935058285399
4.663335508350479
3.76596831997919
5.0589066117011985
11.654580939453396
4.085419545577364
23.762747995

In [ ]:
# prompt: why does no_grid_points shows NaN

import numpy as np
import pandas as pd
from netCDF4 import Dataset
from datetime import datetime, timedelta

# ... (Your existing code) ...

for index, row in catalog_data.iterrows():
    D_eps_value = row['D_eps']
    lat_grid_value = row['lat_grid']
    lon_grid_value = row['lon_grid']
    no_grid_points = D_eps_value / 50

    # Use .loc with boolean indexing and .any() to handle potential missing values
    if grace_data[(grace_data['latitude'] == lat_grid_value) & (grace_data['longitude'] == lon_grid_value)].any().any():
      grace_data.loc[(grace_data['latitude'] == lat_grid_value) & (grace_data['longitude'] == lon_grid_value), 'no_grid_points'] = no_grid_points
    else:
      print(f"No matching grid point found for latitude: {lat_grid_value}, longitude: {lon_grid_value}")

In [32]:
grace_data

,year_month,latitude,longitude,lwe_thickness,no_grid_points
0,2002-04,31.25,-124.75,-0.967506,NaN
1,2002-04,31.25,-124.25,-0.967506,NaN
2,2002-04,31.25,-123.75,-1.035457,NaN
3,2002-04,31.25,-123.25,-1.035457,NaN
4,2002-04,31.25,-122.75,-1.035457,NaN
...,...,...,...,...,...
111799,2024-03,41.75,-116.25,-5.165121,NaN
111800,2024-03,41.75,-115.75,-4.416625,NaN
111801,2024-03,41.75,-115.25,-3.472987,NaN
111802,2024-03,41.75,-114.75,-2.648243,NaN


In [ ]:
# prompt: add a new column value to a dataframe for every iteration

    # Add a new column 'no_grid_points' to the grace_data DataFrame
    grace_data['no_grid_points'] = 0  # Initialize the column with zeros

    # Assign the calculated value to the corresponding rows in the 'no_grid_points' column
    grace_data.loc[(grace_data['latitude'] == lat_grid_value) & (grace_data['longitude'] == lon_grid_value), 'no_grid_points'] = no_grid_points

In [23]:
new_grace_data


,year_month,latitude,longitude,lwe_thickness
143,2002-04,34.25,-119.25,3.046476
627,2002-05,34.25,-119.25,1.353377
1111,2002-08,34.25,-119.25,3.164926
1595,2002-09,34.25,-119.25,6.245145
2079,2002-10,34.25,-119.25,1.549472
...,...,...,...,...
109527,2023-11,34.25,-119.25,-38.934109
110011,2023-12,34.25,-119.25,-39.601843
110495,2024-01,34.25,-119.25,-33.456011
110979,2024-02,34.25,-119.25,-28.251297


In [19]:
grace_data

,year_month,latitude,longitude,lwe_thickness,distance
0,2002-04,31.25,-124.75,-0.967506,6.264982
1,2002-04,31.25,-124.25,-0.967506,5.830952
2,2002-04,31.25,-123.75,-1.035457,5.408327
3,2002-04,31.25,-123.25,-1.035457,5.000000
4,2002-04,31.25,-122.75,-1.035457,4.609772
...,...,...,...,...,...
111799,2024-03,41.75,-116.25,-5.165121,8.077747
111800,2024-03,41.75,-115.75,-4.416625,8.276473
111801,2024-03,41.75,-115.25,-3.472987,8.500000
111802,2024-03,41.75,-114.75,-2.648243,8.746428


In [15]:
# prompt: show all the nieghboring cells you have selected and their lat and lon

# Iterate through each seismic event in the catalog
for index, row in catalog_data.iterrows():
    D_eps_value = row['D_eps']
    lat_grid_value = row['lat_grid']
    lon_grid_value = row['lon_grid']

    # Find the corresponding cells in the mascon data
    # Assuming grace_data DataFrame has 'latitude', 'longitude', and 'lwe_thickness' columns
    # You need to define a suitable distance criteria based on D_eps

    # Calculate distance between earthquake epicenter and each mascon grid cell
    grace_data['distance'] = np.sqrt((grace_data['latitude'] - lat_grid_value)**2 + (grace_data['longitude'] - lon_grid_value)**2)

    # Select mascon cells within the D_eps radius
    cells_within_D_eps = grace_data[grace_data['distance'] <= D_eps_value]

    # Now you have the mascon cells (rows in cells_within_D_eps) that are within D_eps of the earthquake
    print(f"For D_eps = {D_eps_value}, lat = {lat_grid_value}, lon = {lon_grid_value}")
    print(cells_within_D_eps[['latitude', 'longitude']]) # Print latitude and longitude of neighboring cells
    # Further process cells_within_D_eps, e.g., calculate average lwe thickness

For D_eps = 171.8172116081906, lat = 40.75, lon = -124.75
        latitude  longitude
0          31.25    -124.75
1          31.25    -124.25
2          31.25    -123.75
3          31.25    -123.25
4          31.25    -122.75
...          ...        ...
111799     41.75    -116.25
111800     41.75    -115.75
111801     41.75    -115.25
111802     41.75    -114.75
111803     41.75    -114.25

[111804 rows x 2 columns]
For D_eps = 645.1588612290603, lat = 35.75, lon = -121.25
        latitude  longitude
0          31.25    -124.75
1          31.25    -124.25
2          31.25    -123.75
3          31.25    -123.25
4          31.25    -122.75
...          ...        ...
111799     41.75    -116.25
111800     41.75    -115.75
111801     41.75    -115.25
111802     41.75    -114.75
111803     41.75    -114.25

[111804 rows x 2 columns]
For D_eps = 210.60402482250527, lat = 38.25, lon = -118.75
        latitude  longitude
0          31.25    -124.75
1          31.25    -124.25
2          31.2

In [10]:
grace_data

,year_month,latitude,longitude,lwe_thickness
0,2002-04,31.25,-124.75,-0.967506
1,2002-04,31.25,-124.25,-0.967506
2,2002-04,31.25,-123.75,-1.035457
3,2002-04,31.25,-123.25,-1.035457
4,2002-04,31.25,-122.75,-1.035457
...,...,...,...,...
111799,2024-03,41.75,-116.25,-5.165121
111800,2024-03,41.75,-115.75,-4.416625
111801,2024-03,41.75,-115.25,-3.472987
111802,2024-03,41.75,-114.75,-2.648243


In [ ]:
# Approach 2

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
!pip install netCDF4


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the catalog data from Google Drive
file_path = '/content/drive/MyDrive/grace_data/catalog.txt'
catalog_data = pd.read_csv(file_path, delim_whitespace=True)

# OLD CODE START

# OLD CODE END

# New Step 1: Convert the date column to datetime format
catalog_data['date'] = pd.to_datetime(catalog_data['date'], format='%Y-%m-%d')

# Filter seismic data starting from 2022-04
catalog_data = catalog_data[catalog_data['date'] >= '2002-04-01']
catalog_data = catalog_data[catalog_data['mag'] >= 3]

# New Step 2: Divide the dataset into a grid of 0.5 x 0.5 degree
# Round the latitude and longitude to the nearest 0.25 or 0.5
catalog_data['lat_grid'] = np.where((catalog_data['latitude'] % 1) < 0.5,
                                    np.floor(catalog_data['latitude']) + 0.25,
                                    np.floor(catalog_data['latitude']) + 0.75)

catalog_data['lon_grid'] = np.where((catalog_data['longitude'] % 1) < 0.5,
                                     np.floor(catalog_data['longitude']) + 0.25,
                                     np.floor(catalog_data['longitude']) + 0.75)



# New Step 3: Group by lat_grid, lon_grid, and month to get the sum of events per month
catalog_data['year_month'] = catalog_data['date'].dt.to_period('M')
events_per_month = catalog_data.groupby(['lat_grid', 'lon_grid', 'year_month']).agg(
    event_count=('year_month', 'size'),
    lat_list=('latitude', lambda x: list(x)),
    lon_list=('longitude', lambda x: list(x)),
    max_mag=('mag', 'max'),
    min_mag=('mag', 'min'),
    avg_mag=('mag', 'mean')
).reset_index()

# New Step 4: Sort the result by year_month
events_per_month = events_per_month.sort_values(by='year_month')

# Step 5: Count the number of unique grid points for each month
unique_grid_points_per_month = events_per_month.groupby('year_month')[['lat_grid', 'lon_grid']].nunique().reset_index()
unique_grid_points_per_month.rename(columns={'lat_grid': 'unique_lat_grid_count', 'lon_grid': 'unique_lon_grid_count'}, inplace=True)

# Display the number of unique grid points for each month
print("Number of unique grid points for each month:")
print(unique_grid_points_per_month)


# Display the result
events_per_month.head()

# New Step 7: Read a NetCDF file (GRACE Mascon data)
from netCDF4 import Dataset

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the GRACE Mascon data from Google Drive
netcdf_file_path = '/content/drive/MyDrive/grace_data/GRACE_Mascon_data.nc'
with Dataset(netcdf_file_path, mode='r') as nc_file:
    # List all variables in the NetCDF file
    print("Variables in GRACE Mascon NetCDF file:")
    print(nc_file.variables.keys())

    # Extract data for specific variables (example: lwe_thickness)
    lwe_thickness = nc_file.variables['lwe_thickness'][:]
    latitudes = nc_file.variables['lat'][:]
    longitudes = nc_file.variables['lon'][:]

    # Convert longitudes from [0, 360] to [-180, 180] range
    longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
    times = nc_file.variables['time'][:]

    # Display basic information about the extracted data
    print("LWE Thickness Data Shape:", lwe_thickness.shape)
    print("Latitude Data Shape:", latitudes.shape)
    print("Longitude Data Shape:", longitudes.shape)
    print("Time Data Shape:", times.shape)

    # Step 8: Add year_month to GRACE data
from datetime import datetime, timedelta

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Step 8: Filter GRACE data for California region only
california_lat_min, california_lat_max = 31, 42
california_lon_min, california_lon_max = -125, -114

# Filter GRACE data for California latitudes and longitudes
lat_indices = np.where((latitudes >= california_lat_min) & (latitudes <= california_lat_max))[0]
lon_indices = np.where((longitudes >= california_lon_min) & (longitudes <= california_lon_max))[0]

# Extract the filtered data
filtered_latitudes = latitudes[lat_indices]
filtered_longitudes = longitudes[lon_indices]
filtered_lwe_thickness = lwe_thickness[:, lat_indices, :][:, :, lon_indices]

# Convert time to datetime format and create year_month column
time_reference = datetime(2002, 1, 1)  # Assuming GRACE times start from January 2002
grace_dates = [time_reference + timedelta(days=int(t)) for t in times]
grace_year_month = pd.to_datetime(grace_dates).to_period('M')

# Create a DataFrame for filtered GRACE data
filtered_grace_data_list = []
for time_idx, year_month in enumerate(grace_year_month):
    for lat_idx, latitude in enumerate(filtered_latitudes):
        for lon_idx, longitude in enumerate(filtered_longitudes):
            lwe_value = filtered_lwe_thickness[time_idx, lat_idx, lon_idx]
            filtered_grace_data_list.append({
                'year_month': year_month,
                'latitude': latitude,
                'longitude': longitude,
                'lwe_thickness': lwe_value
            })

# Convert list of dictionaries to DataFrame
grace_data = pd.DataFrame(filtered_grace_data_list)

# Ensure correct column names for grace_data
# No need to rename columns, keep as 'latitude' and 'longitude'

# Step 9: Display columns for debugging
print("GRACE Data Columns:")
print(grace_data.columns.tolist())
print("Events per Month Columns:")
print(events_per_month.columns.tolist())
print(events_per_month.columns)

# Step 9: Based on the printed column names, adjust the merge operation
grace_filtered_data = grace_data.merge(events_per_month,
                                       left_on=['latitude', 'longitude', 'year_month'],
                                       right_on=['lat_grid', 'lon_grid', 'year_month'],
                                       how='inner')

# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness']]
print("Filtered GRACE Data:")
print(grace_filtered_df)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
Mounted at /content/drive


<ipython-input-5-336d50d7b15a>:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  catalog_data = pd.read_csv(file_path, delim_whitespace=True)


Number of unique grid points for each month:
    year_month  unique_lat_grid_count  unique_lon_grid_count
0      2002-04                      7                      8
1      2002-05                     12                     10
2      2002-06                      8                      9
3      2002-07                     10                     12
4      2002-08                      9                     10
..         ...                    ...                    ...
256    2023-08                     12                     14
257    2023-09                     10                     11
258    2023-10                     12                     12
259    2023-11                     10                     10
260    2023-12                     10                      7

[261 rows x 3 columns]
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Variables in GRACE Mascon NetCDF file:
dict_keys(['lon', 'lat', 'time'

In [ ]:
grace_filtered_data

In [ ]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [ ]:
!pip install keras_tuner
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.LSTM(n_neurons, return_sequences=True, input_shape=[1, 3] , activation = "tanh"))
    for _ in range(n_hidden):
        return_sequences = True if _ < n_hidden - 1 else False
        model.add(tf.keras.layers.LSTM(n_neurons, return_sequences=return_sequences, activation="tanh"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss="mse", optimizer=optimizer, # Changed loss to 'mse' for regression
                  metrics=["mae"]) # Changed metric to 'mae' for regression
    return model

    """
    tf.keras.layers.LSTM(n_neurons, return_sequences=True, input_shape=[None, 3] , activation = "tanh"),
    for _ in range(n_hidden):
        model.add(tf.keras.layers.LSTM(n_neurons, return_sequences=True, activation="tanh"))
    model.add(tf.keras.layers.Dense(1));
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model
    """

In [ ]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs,
                        callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae

In [ ]:
# Display the filtered GRACE Data
grace_filtered_df = grace_filtered_data[['latitude', 'longitude', 'year_month', 'lwe_thickness', 'event_count', 'max_mag', 'min_mag', 'avg_mag']]


# Step 10: Prepare data for LSTM model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Extract predictors and target
X = grace_filtered_df[['event_count', 'year_month','max_mag', 'min_mag', 'avg_mag','lwe_thickness']]
y = grace_filtered_df[['year_month','lwe_thickness']]

X = X.set_index('year_month')

X_train = X.loc['2002-04':'2013-05', ['event_count', 'max_mag', 'avg_mag']]
X_valid = X.loc['2013-06':'2020-05', ['event_count', 'max_mag', 'avg_mag']]
X_test = X.loc['2020-06':'2024-01', ['event_count', 'max_mag', 'avg_mag']]

y_train = X.loc['2002-04':'2013-05', 'lwe_thickness']
y_valid = X.loc['2013-06':'2020-05', 'lwe_thickness']

tf.random.set_seed(42)
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    X_train.to_numpy(),
    targets=y_train,
    sequence_length=1,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    X_valid.to_numpy(),
    targets=y_valid,
    sequence_length=1,
    batch_size=32
)

random_search_tuner = kt.RandomSearch(
    build_model,
    objective="val_mae",
    max_trials=10,
    overwrite=True,
    directory="/content/drive/MyDrive/grace_data/tuner_srch",
    project_name="hp_search"
)

rst = random_search_tuner.search(train_ds, epochs=10, validation_data=valid_ds)

best_hp = random_search_tuner.get_best_hyperparameters()[0]
model = random_search_tuner.hypermodel.build(best_hp)
"""
deep_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 3]),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1)
])
"""

fit_and_evaluate(model, train_ds , valid_ds , 1e-4 )

#,'max_mag', 'min_mag', 'avg_mag','lwe_thickness'

In [ ]:
model.summary()

In [ ]:
X_test_reshaped = X_test.to_numpy().reshape(X_test.shape[0], 1, X_test.shape[1])


In [ ]:
X_test

In [ ]:
best_hp.values

In [2]:
# prompt: convert X_test_reshaped to a dataframe with first col as   event_count , second col as max_mag and third col as avg_mag. show first 10 rows

import pandas as pd

# Assuming X_test_reshaped is already defined as in your provided code
# X_test_reshaped = X_test.to_numpy().reshape(X_test.shape[0], 1, X_test.shape[1])

# Create a DataFrame from X_test_reshaped
X_test_df = pd.DataFrame(X_test_reshaped.reshape(X_test.shape[0], X_test.shape[1]), columns=['event_count', 'max_mag', 'avg_mag'])

# Display the first 10 rows
print(X_test_df.head(10))

NameError: name 'X_test_reshaped' is not defined

In [3]:
# prompt: make a df from model.predict(X_test_reshaped) and name the col as lwe. show first 10 rows

# Assuming 'model' and 'X_test_reshaped' are defined as in your previous code.
import pandas as pd

predictions = model.predict(X_test_reshaped)
# Create a DataFrame from the predictions
df_predictions = pd.DataFrame(predictions, columns=['lwe'])
# Show the first 10 rows
print(df_predictions.head(10))

NameError: name 'model' is not defined

In [4]:
model.predict(X_test_reshaped)

NameError: name 'model' is not defined